In [24]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool


In [9]:
df = pd.read_parquet("/Users/riyaazkhan/Documents/Imperial_Maths/Algorithmic_Trading_Club/Projects/Horse-Racing-Project/Data/Processed/XGBoost_Dataset.v2_trimmed.parquet")

In [10]:
useful_features = ['year','date','father','mother','gfather','horseName','jockeyName','trainerName','res_win','betting_prob_win','OR_Mean_Last3_Z_Race_Rank', 'rpr_Last_Race_Rank', 'racers_per_race', 'rpr_Last_Z_Race_Rank', 'rpr_Mean_Last3_Z_Race_Rank', 'OR_Mean_Last3_Race_Rank', 'course_clean', 'OR_best_last_5', 'trainer_win_encoded', 'TR_Mean_Last3_Z_Race_Rank', 'prize', 'rpr_last_mean_in_race', 'trainer_jumptype_win_Rate', 'jockey_jumptype_win_Rate', 'rpr_best_last_5', 'jockey_win_encoded', 'TR_best_last_5', 'rpr_Mean_Last3_Race_Rank', 'diff_OR_RPR_last_3', 'OR_Mean_Last3_Race_std', 'rpr_Mean_Last3_Race_std', 'prev_max_rpr', 'TR_Mean_Last3_Race_Rank', 'TR_Last_Race_Rank', 'weight_rank', 'diff_OR_RPR_last', 'prev_rpr_diff_from_top', 'rpr_Last_Race_std', 'age_rel_min', 'horse_jumptype_win_Rate_delta', 'trainer_class_winrate', 'weight_difference_to_top', 'rpr_last', 'trainer_going_winrate', 'original_index', 'previous_weight', 'days_since_last_race', 'OR_mean_last_3_median_in_race', 'trainer_course_win_Rate', 'horse_win_encoded', 'jockey_going_winrate', 'jockey_course_win_Rate', 'jockey_distanceband_winrate', 'TR_Mean_Last3_Race_std', 'rpr_Last_Race_iqr', 'rclass_change_numeric', 'saddle_rank_norm', 'horse_jumptype_win_Rate', 'distance', 'rpr_last_median_in_race', 'rpr_mean_last_3_median_in_race', 'days_since_last', 'OR_Last_Z_Race_Rank', 'prev_OR_diff_from_top', 'OR_mean_last_3_mean_in_race', 'father_going_winrate', 'jockey_distanceband_racecount', 'headGear', 'father_distanceband_winrate', 'weight_difference_to_median', 'headgear_count', 'countryCode', 'trainer_course_racecount', 'father_coursesurface_winrate', 'horse_coursesurface_win_Rate_delta', 'weight_differnce_last_race', 'horse_distanceband_win_Rate_delta', 'gfather_jumptype_winrate', 'TR_Last_Race_std', 'horse_going_winrate', 'rclass_numeric', 'rpr_mean_last_3_mean_in_race', 'OR_last', 'rpr_mean_last_3', 'saddle_rank', 'horse_bandbin_racecount', 'OR_Last_Race_Rank', 'horse_distanceband_racecount', 'father_jumptype_winrate', 'trainer_jumptype_racecount', 'mother_jumptype_winrate', 'horse_jockey_win_Rate_delta', 'weight', 'TR_mean_last_3_median_in_race', 'OR_mean_last_3', 'horse_jockey_win_Rate', 'horse_course_winrate', 'TR_Last_Z_Race_Rank', 'father_distanceband_racecount', 'prev_max_TR', 'TR_last_mean_in_race', 'OR_mean_last_20', 'horse_jumptype_racecount', 'trainer_saddlerankbin_winrate', 'jockey_jumptype_racecount', 'band_upper', 'horse_bandbin_win_Rate_delta', 'horse_course_winrate_delta', 'mother_course_winrate', 'trainer_saddlerankbin_racecount', 'father_class_winrate', 'horse_coursesurface_win_Rate', 'father_band_racecount', 'race_min_age', 'horse_coursesurface_racecount', 'horse_bandbin_win_Rate', 'jockey_distanceband_win_Rate_delta', 'TR_Last_Race_iqr', 'horse_distanceband_win_Rate', 'gfather_jumptype_racecount', 'father_course_winrate', 'horse_going_winrate_delta', 'OR_last_median_in_race', 'race_month', 'gfather_course_racecount', 'TR_mean_last_3_mean_in_race', 'prev_max_OR', 'mother_jumptype_racecount', 'trainer_class_racecount', 'horse_class_racecount', 'trainer_going_winrate_delta', 'jockey_going_racecount', 'jockey_course_win_Rate_delta', 'OR_Last_Race_std', 'TR_last_median_in_race', 'rpr_mean_last_20', 'best_dist_band_hist', 'father_coursesurface_racecount', 'gfather_course_winrate']


In [31]:
cat_cols = ['horseName','jockeyName','trainerName','course_clean','father',
            'mother','gfather',
           'countryCode','headGear','prev_headgear','course_surface','track_layout',
            'race_jump_type','best_dist_band_hist']


In [11]:
def keep_cols_strict(df, cols_to_keep, *, copy=True):
    missing = [c for c in cols_to_keep if c not in df.columns]
    if missing:
        raise KeyError(f"Missing columns: {missing}")
    return df.loc[:, cols_to_keep].copy() if copy else df.loc[:, cols_to_keep]


In [12]:
df = keep_cols_strict(df, useful_features)

In [13]:
tr_data = df[df['year'] <= 2014]
val_data = df[(df['year'] < 2018) & (df['year'] > 2014)]
bkt_data = df[df['year'] >= 2018]

In [14]:
tr_data = tr_data.copy().sort_values('date')
val_data = val_data.copy().sort_values('date')
bkt_data = bkt_data.copy().sort_values('date')

tr_data = tr_data.drop(columns='date')
val_data = val_data.drop(columns='date')
bkt_data = bkt_data.drop(columns='date')

In [15]:
tr_data_y = tr_data['res_win']
tr_data_p = tr_data['betting_prob_win']
tr_data_x = tr_data.drop(columns=['res_win','betting_prob_win'])

val_data_y = val_data['res_win']
val_data_p = val_data['betting_prob_win']
val_data_x = val_data.drop(columns=['res_win','betting_prob_win'])

bkt_data_y = bkt_data['res_win']
bkt_data_p = bkt_data['betting_prob_win']
bkt_data_x = bkt_data.drop(columns=['res_win','betting_prob_win'])

In [32]:
for c in cat_cols:
    if c in tr_data_x.columns:
        tr_data_x.loc[:, c]  = tr_data_x[c].astype('string').str.strip().fillna('__MISSING__')
        val_data_x.loc[:, c] = val_data_x[c].astype('string').str.strip().fillna('__MISSING__')
        
cat_idx = [tr_data_x.columns.get_loc(c) for c in cat_cols if c in tr_data_x.columns]

In [33]:
pool_tr = Pool(tr_data_x, tr_data_y, cat_features=cat_idx)
pool_va = Pool(val_data_x, val_data_y, cat_features=cat_idx)

In [37]:
from catboost.utils import get_gpu_device_count
get_gpu_device_count()

0

In [38]:
cb = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='Logloss',
    custom_metric=['AUC','BrierScore'],

    iterations=1200,          # much fewer rounds
    learning_rate=0.10,       # faster learning (compensates for fewer iterations)
    depth=6,                  # shallower trees = faster
    l2_leaf_reg=5,            # mild regularization

    bootstrap_type='Bernoulli',
    subsample=0.8,            # row subsampling
    rsm=0.7,                  # column subsampling (feature subsample)

    border_count=32,          # fewer splits for speed on CPU
    random_strength=1.0,

    od_type='Iter',
    od_wait=100,              # earlier early-stop
    use_best_model=True,

    thread_count=-1,
    verbose=100,
    task_type='CPU'
)

cb.fit(pool_tr, eval_set=pool_va)


0:	learn: 0.5963593	test: 0.6031795	best: 0.6031795 (0)	total: 5.64s	remaining: 1h 52m 41s
100:	learn: 0.2473189	test: 0.2837420	best: 0.2837420 (100)	total: 10m 25s	remaining: 1h 53m 28s
200:	learn: 0.2142529	test: 0.2787580	best: 0.2787580 (200)	total: 23m 25s	remaining: 1h 56m 23s
300:	learn: 0.1939094	test: 0.2770990	best: 0.2770520 (298)	total: 36m 34s	remaining: 1h 49m 14s
400:	learn: 0.1841374	test: 0.2757947	best: 0.2755650 (397)	total: 50m 13s	remaining: 1h 40m 4s
500:	learn: 0.1774641	test: 0.2750382	best: 0.2746108 (472)	total: 1h 1m 18s	remaining: 1h 25m 32s


KeyboardInterrupt: 

In [ ]:
cb.save_model("catboost_model.cbm")